# New Section

In [6]:
import pandas as pd
import sqlite3

In [9]:
import os
os.listdir()


['.config', 'sample_data']

In [10]:
from google.colab import files
uploaded = files.upload()

Saving users.json to users.json
Saving restaurants.sql to restaurants.sql
Saving orders.csv to orders.csv


In [60]:
import pandas as pd
import sqlite3

orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [61]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [66]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()



,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [67]:
df = orders.merge(users, on="user_id", how="left")
df = df.merge(restaurants, on="restaurant_id", how="left")

df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [68]:
df.to_csv("final_food_delivery_dataset.csv", index=False)


In [19]:
df.to_csv("final_food_delivery_dataset.csv", index=False)


In [69]:
df = df.rename(columns={"restaurant_name_x": "restaurant_name"})
df = df.drop(columns=["restaurant_name_y"])
df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Italian,3.0


In [70]:
df.to_csv("final_food_delivery_dataset.csv", index=False)


In [71]:
df[df["membership"]=="Gold"].groupby("city")["total_amount"].sum().sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [72]:
df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [74]:
user_sum = df.groupby("user_id")["total_amount"].sum()
user_sum[user_sum > 1000].count()



np.int64(2544)

In [84]:
from google.colab import files
files.download("final_food_delivery_dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [78]:
df["rating_range"] = pd.cut(df["rating"], bins=[3,3.5,4,4.5,5])
df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


/tmp/ipython-input-3428258696.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


,total_amount
rating_range,
"(4.5, 5.0]",2197030.75
"(4.0, 4.5]",1960326.26
"(3.0, 3.5]",1881754.57
"(3.5, 4.0]",1717494.41


In [80]:
df[df["membership"]=="Gold"].groupby("city")["total_amount"].mean().sort_values(ascending=False)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [81]:
df.groupby("cuisine").agg(
    restaurants=("restaurant_id","nunique"),
    revenue=("total_amount","sum")
).sort_values("restaurants")



,restaurants,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [82]:
(df[df["membership"]=="Gold"].shape[0] / df.shape[0]) * 100



49.87

In [83]:
rest_stats = df.groupby("restaurant_name").agg(
    avg_value=("total_amount","mean"),
    count=("order_id","count")
)

rest_stats[rest_stats["count"] < 20].sort_values("avg_value", ascending=False).head()


,avg_value,count
restaurant_name,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19


In [36]:
df.groupby(["membership","cuisine"])["total_amount"].sum().sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [37]:
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)
df["quarter"] = df["order_date"].dt.to_period("Q")

df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [44]:
df[df["membership"]=="Gold"].groupby("city")["total_amount"].mean().sort_values(ascending=False)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [45]:
df.groupby("cuisine").agg(
    restaurants=("restaurant_id","nunique"),
    revenue=("total_amount","sum")
).sort_values("restaurants")


,restaurants,revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [52]:
df[df["membership"]=="Gold"].shape[0]


4987

In [53]:
hy_rev = df[df["city"]=="Hyderabad"]["total_amount"].sum()
round(hy_rev)


1889367

In [54]:
df["user_id"].nunique()


2883

In [55]:
gold_avg = df[df["membership"]=="Gold"]["total_amount"].mean()
round(gold_avg, 2)


np.float64(797.15)

In [56]:
df[df["rating"] >= 4.5].shape[0]


3374

In [57]:
gold_city = df[df["membership"]=="Gold"].groupby("city")["total_amount"].sum().sort_values(ascending=False)
gold_city


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [58]:
df[(df["membership"]=="Gold") & (df["city"]=="Chennai")].shape[0]


1337

In [59]:
df.shape[0]


10000